<a href="https://colab.research.google.com/github/Neel7317/NLP/blob/main/Text_Intent_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1OlcvGWReJMuyYQuOZm149vHWwPtlboR6
To: /content/train.csv
100% 799k/799k [00:00<00:00, 6.15MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w
To: /content/valid.csv
100% 43.3k/43.3k [00:00<00:00, 23.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF
To: /content/test.csv
100% 43.1k/43.1k [00:00<00:00, 2.16MB/s]


In [3]:
import pandas as pd

In [4]:
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")

In [5]:
data=train
data=data.append(test).reset_index(drop=True)
data=data.append(valid).reset_index(drop=True)

In [6]:
data.head()

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist


In [8]:
labelencode={
    'PlayMusic':0,
    'GetWeather':1,
    'BookRestaurant':2,
    'SearchScreeningEvent':3,
    'RateBook':4,
    'SearchCreativeWork':5,
    'AddToPlaylist':6
}

In [9]:
data['labelintent']=data.intent.map(labelencode)

In [10]:
data.head()

,text,intent,labelintent
0,listen to westbam alumb allergic on google music,PlayMusic,0
1,add step to me to the 50 clásicos playlist,AddToPlaylist,6
2,i give this current textbook a rating value of...,RateBook,4
3,play the song little robin redbreast,PlayMusic,0
4,please add iris dement to my playlist this is ...,AddToPlaylist,6


In [11]:
data.tail()

,text,intent,labelintent
14479,is any cinema playing the spirit of youth,SearchScreeningEvent,3
14480,what are the movie times for animated movies i...,SearchScreeningEvent,3
14481,what s the movie schedule at great escape thea...,SearchScreeningEvent,3
14482,show the times for cheers for miss bishop at d...,SearchScreeningEvent,3
14483,i want to see married to the enemy 2 at a cinema,SearchScreeningEvent,3


In [12]:
from keras.utils.np_utils import to_categorical

In [13]:
y_cat=to_categorical(data['labelintent'])

In [14]:
y_cat[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], y_cat, test_size=0.20)

In [18]:
x_train[:5]

9094     i want to send music to ann s lists from sir j...
5331          what is the weather close-by my current spot
14404               i want animated movies at mjr theatres
11822         out of 6 stars the last novel gets only four
5755                         rate the current novel a zero
Name: text, dtype: object

In [19]:
y_train[:5]

array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [20]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 3.9 MB/s 
     |████████████████████████████████| 462 kB 55.8 MB/s 


In [21]:
import tensorflow_hub as hub
import tensorflow_text as text

In [22]:
preprocess=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [23]:
import tensorflow as tf

In [24]:
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocess(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(7, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [26]:
model.compile(optimizer='adam',loss='CategoricalCrossentropy',metrics=['accuracy'])

In [29]:
model.fit(x_train,y_train,validation_split=0.1,epochs=5,batch_size=32,shuffle=True)

Epoch 1/5
326/326 [==============================] - 4433s 14s/step - loss: 1.3204 - accuracy: 0.5664 - val_loss: 1.0489 - val_accuracy: 0.6997
Epoch 2/5
326/326 [==============================] - 4446s 14s/step - loss: 0.9684 - accuracy: 0.7089 - val_loss: 0.7825 - val_accuracy: 0.8050
Epoch 3/5
326/326 [==============================] - 4489s 14s/step - loss: 0.7966 - accuracy: 0.7692 - val_loss: 0.6598 - val_accuracy: 0.8404
Epoch 4/5
326/326 [==============================] - 4494s 14s/step - loss: 0.6975 - accuracy: 0.7975 - val_loss: 0.5851 - val_accuracy: 0.8550
Epoch 5/5
326/326 [==============================] - 4465s 14s/step - loss: 0.6334 - accuracy: 0.8185 - val_loss: 0.5184 - val_accuracy: 0.8749


In [35]:
a=model.predict(['play the song little robin redbreast'])

In [36]:
a.argmax()

0

In [39]:
model.save('my_model.h5')

In [43]:
b=model.predict(['Rate this book as awful'])

In [44]:
b.argmax()

4

In [45]:
model.evaluate(x_test,y_test)

91/91 [==============================] - 1159s 13s/step - loss: 0.5423 - accuracy: 0.8602


[0.5423306822776794, 0.8602002263069153]

In [46]:
y_pred=model.predict(x_test)

In [48]:
model.save("text_classifier_v1")

INFO:tensorflow:Assets written to: text_classifier_v1/assets


INFO:tensorflow:Assets written to: text_classifier_v1/assets


In [53]:
from tensorflow import keras

# load model
new_model = keras.models.load_model("text_classifier_v1")



In [54]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [55]:
c=new_model.predict(['rate this book as awful'])

In [56]:
c.argmax()

4

In [58]:
 nn=tf.keras.models.load_model("my_model.h5")

ValueError: ignored

In [59]:
path="/content/my_model.h5"

In [60]:
my_reloaded_model = tf.keras.models.load_model((path),custom_objects={'KerasLayer':hub.KerasLayer})

In [63]:
dd=my_reloaded_model.predict(['add step to me to the 50 clásicos playlist'])


In [64]:
dd.argmax()

6

In [49]:
import numpy as np


In [50]:
y_pred=np.argmax(y_pred, axis=1)

In [52]:
y_pred[:5]

array([5, 3, 3, 4, 6])